In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TIMESTEPS = 1500
LEAD_COUNT = 12

EPOCHS=25
BATCH_SIZE=32
LEARNING_RATE=1e-5

In [3]:
dataset = torch.load("vars/dataset2.pt")

split_size = list()
split_size.append(int(0.64*len(dataset)))
split_size.append(int(0.16*len(dataset)))
split_size.append(len(dataset)-sum(split_size))

datasets = torch.utils.data.random_split(dataset, split_size)

In [4]:
dataloaders = [torch.utils.data.DataLoader(i, batch_size=b, shuffle=True) for i, b in zip(datasets, [BATCH_SIZE, ]+split_size[1:])]

In [12]:
class ECGNN(nn.Module):
    def __init__(self):
        super(ECGNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 16, (3, 100), padding=1),
            nn.Conv2d(16, 16, (3, 100), padding=1),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, (3, 100), padding=1),
            nn.Conv2d(32, 32, (3, 100), padding=1),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, (3, 100))
        )
        self.lin_block = nn.Sequential(
            nn.Linear(4160, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )
    
    def forward(self, x):
        x = self.conv_block(x)
        x = torch.flatten(x, 1)
        x = self.lin_block(x)
        x = torch.sigmoid(x)
        return x

In [6]:
def train(model, loss, maeloss, mseloss, optimizer, data):
    global ypred
    global x
    nbat = len(data)
    aloss, aacc, amloss, amsloss = 0, 0, 0, 0
    for batch, (x, y) in enumerate(data):
        x = x.to(device = DEVICE, dtype=torch.float)
        y = y.to(device = DEVICE, dtype=torch.float)
        ypred = model(x)
        bloss = loss(ypred ,y)
        
        optimizer.zero_grad()
        bloss.backward()
        optimizer.step()

        with torch.no_grad():
            acc = ((ypred.squeeze()>=0.50).int()==y).sum().item()/BATCH_SIZE

            bloss = loss(ypred, y)
            mloss = maeloss(ypred, y).item()
            msloss = mseloss(ypred, y).item()
            amloss += mloss
            amsloss += msloss
            aloss += bloss.item()
            aacc += acc


        print("Train - crossentropy_loss: %f  mae_loss:%f  mse_loss: %f  accuracy: %f  [%d/%d]"%(bloss.item(), mloss, msloss, acc, batch+1, nbat), end="\r")
    aloss /= nbat
    aacc /= nbat
    amloss /= nbat
    amsloss /= nbat
    print("\nTrain - crossentropy_loss: %f  mae_loss: %f  mse_loss: %f  accuracy: %f  [Average]"%(aloss, amloss, amsloss, aacc))
    return (aloss, amloss, amsloss, aacc)

In [7]:
def test(model, loss, maeloss, mseloss, data):
    nbat = len(data)
    acc, bloss, mloss, msloss = 0, 0, 0, 0

    with torch.no_grad():
        for x, y in data:
            x = x.to(device=DEVICE, dtype=torch.float)
            y = y.to(device=DEVICE, dtype=torch.float)
            ypred = model(x)
            bloss += loss(ypred ,y)
            mloss += maeloss(ypred ,y).item()
            msloss += mseloss(ypred, y).item()
            acc += ((ypred.squeeze()>=0.50).int()==y).sum().item()/BATCH_SIZE
    
    acc /= nbat
    bloss /= nbat
    mloss /= nbat
    msloss /= nbat
    print("Test - crossentropy_loss: %f  mae_loss: %f  mse_loss: %f  accuracy: %f"%(bloss.item(), mloss, msloss, acc))
    return (bloss.item(), mloss, r2, msloss, acc)

In [8]:
def fit(model, loss, mloss, msloss, optimizer, train_loader, test_loader, epochs):
    train_history = {
        "epochs": list(range(1, epochs+1)),
        "crossentropy_loss": list(),
        "mae_loss": list(),
        "mse_loss": list(),
        "accuracy": list()
    }
    test_history = {
        "epochs": list(range(1, epochs+1)),
        "crossentropy_loss": list(),
        "mae_loss": list(),
        "mse_loss": list(),
        "accuracy": list()
    }
    for e in range(epochs):
        print("EPOCH %d/%d"%(e+1, epochs))

        l, m, ms, acc = train(model, loss, mloss, msloss, optimizer, train_loader)
        train_history["crossentropy_loss"].append(l)
        train_history["mae_loss"].append(m)
        train_history["mse_loss"].append(ms)
        train_history["accuracy"].append(acc)

        l, m, ms, acc = test(model, loss, mloss, msloss, test_loader)
        test_history["crossentropy_loss"].append(l)
        test_history["mae_loss"].append(m)
        test_history["mse_loss"].append(ms)
        test_history["accuracy"].append(acc)

        print("\n--------------------\n")
    return {
        "train": train_history,
        "test": test_history
    }

In [13]:
model = ECGNN().to(DEVICE)
summary(model, (1, 12, 1500))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 12, 1403]           4,816
            Conv2d-2         [-1, 16, 12, 1306]          76,816
         MaxPool2d-3           [-1, 16, 6, 653]               0
            Conv2d-4           [-1, 32, 6, 556]         153,632
            Conv2d-5           [-1, 32, 6, 459]         307,232
         MaxPool2d-6           [-1, 32, 3, 229]               0
            Conv2d-7           [-1, 32, 1, 130]         307,232
           Flatten-8                 [-1, 4160]               0
            Linear-9                 [-1, 1024]       4,260,864
             ReLU-10                 [-1, 1024]               0
           Linear-11                  [-1, 512]         524,800
             ReLU-12                  [-1, 512]               0
           Linear-13                    [-1, 1]             513
Total params: 5,635,905
Trainable param

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss = nn.BCELoss()
mloss = nn.L1Loss(reduction="mean")
msloss = nn.MSELoss()

In [15]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [16]:
history = fit(model, loss, mloss, msloss, optimizer, dataloaders[0], dataloaders[1], EPOCHS)

EPOCH 1/25
Train - crossentropy_loss: 0.094351  mae_loss:0.088484  mse_loss: 0.010624  accuracy: 0.156250  [277/277]]
Train - crossentropy_loss: 0.558997  mae_loss: 0.390666  mse_loss: 0.191739  accuracy: 16.175880  [Average]


RuntimeError: CUDA out of memory. Tried to allocate 2.06 GiB (GPU 0; 5.81 GiB total capacity; 2.51 GiB already allocated; 1.55 GiB free; 2.65 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF